<a href="https://colab.research.google.com/github/RtjShreyD/Eng-Mandarin/blob/master/train_v02.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [3]:
!pip install torchvision

     |████████████████████████████████| 734.6MB 25kB/s 
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 20.1MB/s 
     |████████████████████████████████| 133kB 71.3MB/s 
     |████████████████████████████████| 194kB 57.6MB/s 
     |████████████████████████████████| 245kB 57.1MB/s 
     |████████████████████████████████| 163kB 63.0MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 256kB 58.7MB/s 
     |████████████████████████████████| 1.0MB 49.6MB/s 
  Created wheel for overrides: filename=overrides-2.7.0-cp36-none-any.whl size=5600 sha256=46a0ea051af30debf88b32f1550a44bc4501448321cf20f88fa1c4009981ba0d
  Stored in directory: /root/.cache/pip/wheels/8c/7c/ef/80508418b67d87371c5b3de49e03eb22ee7c1d19affb5099f8
  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31894 sha256=adfb8c977911b55ac78605ac955e0f7f6df3bbbec070a3e3e9bed234068af037
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61

In [0]:
import itertools
import os
import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer

EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256

In [46]:
reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=CharacterTokenizer(),
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
    
train_dataset = reader.read('/content/drive/My Drive/Eng_Mandarin/data/tatoeba.eng_cmn.train.tsv')
validation_dataset = reader.read('/content/drive/My Drive/Eng_Mandarin/data/tatoeba.eng_cmn.dev.tsv')

vocab = Vocabulary.from_instances(train_dataset + validation_dataset, 
                                  min_count={'tokens': 3, 'target_tokens': 3})

en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'), 
                         embedding_dim=EN_EMBEDDING_DIM)

37400it [00:12, 2900.89it/s]
4676it [00:01, 3201.16it/s]
100%|██████████| 42076/42076 [00:00<00:00, 66525.86it/s]


In [0]:
encoder = StackedSelfAttentionEncoder(
        input_dim=EN_EMBEDDING_DIM, 
        hidden_dim=HIDDEN_DIM, 
        projection_dim=128, 
        feedforward_hidden_dim=128, 
        num_layers=1, 
        num_attention_heads=8)

source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

In [48]:
attention = DotProductAttention()

max_decoding_steps = 20 

model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)

if torch.cuda.is_available():
  CUDA_DEVICE = 0
  model = model.cuda(CUDA_DEVICE)
  print("Model deployed on GPU")
else:
  CUDA_DEVICE = -1
  print("Model deployed on CPU")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

iterator.index_with(vocab)

Model deployed on GPU


In [49]:
mod_path = '/content/drive/My Drive/Eng_Mandarin/data/tmp_v02.1'
try:   
    if(os.path.exists(mod_path)):
      print("\nModel Path exists move ahead")
    else:
      os.mkdir(mod_path)
except OSError:
    print ("Creation of the directory failed")
else:  
    print ("Successfully created the tmp directory model path")

Successfully created the tmp directory model path


In [50]:
try:
  trainer = Trainer(model=model, 
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=70,
                  cuda_device=CUDA_DEVICE)
                     
  trainer.train()
  print("*******************************Completed 70 epochs***********************************")
  predictor = SimpleSeq2SeqPredictor(model, reader)

except KeyboardInterrupt:
  print("Training interrupted....all weights saved")

for instance in itertools.islice(validation_dataset, 10):
  print('SOURCE:', instance.fields['source_tokens'].tokens)
  print('GOLD:', instance.fields['target_tokens'].tokens)
  print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])
  print('////////////////////////////////////////////\n')

with open(os.path.join(mod_path, "model.th"), 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files(os.path.join(mod_path, "vocabulary"))

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 4.6968 ||: 100%|██████████| 1169/1169 [00:39<00:00, 29.69it/s]
BLEU: 0.0086, loss: 4.1483 ||: 100%|██████████| 147/147 [00:12<00:00, 12.25it/s]
loss: 3.8138 ||: 100%|██████████| 1169/1169 [00:36<00:00, 31.77it/s]
BLEU: 0.0157, loss: 3.6491 ||: 100%|██████████| 147/147 [00:12<00:00, 11.43it/s]
loss: 3.3700 ||: 100%|██████████| 1169/1169 [00:36<00:00, 32.72it/s]
BLEU: 0.0298, loss: 3.3443 ||: 100%|██████████| 147/147 [00:12<00:00, 11.64it/s]
loss: 3.0333 ||: 100%|██████████| 1169/1169 [00:36<00:00, 31.70it/s]
BLEU: 0.0445, loss: 3.1122 ||: 100%|██████████| 147/147 [00:12<00:00, 11.86it/s]
loss: 2.7483 ||: 100%|██████████| 1169/1169 [00:36<00:00, 31.85it/s]
BLEU: 0.0574, loss: 2.9382 ||: 100%|██████████| 147/147 [00:12<00:00, 11.32it/s]
loss: 2.5116 ||: 100%|██████████| 1169/1169 [00:36<00:00, 30.57it/s]
BLEU: 0.0725, loss: 2.8033 ||: 100%|██████████| 147/147 [00:13<00:00, 11.15it

*******************************Completed 70 epochs***********************************
SOURCE: [@start@, I, have, to, go, to, sleep, ., @end@]
GOLD: [@start@, 我, 该, 去, 睡, 觉, 了, 。, @end@]
PRED: ['我', '睡', '觉', '了', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, just, do, n't, know, what, to, say, ., @end@]
GOLD: [@start@, 我, 就, 是, 不, 知, 道, 說, 些, 什, 麼, 。, @end@]
PRED: ['我', '只', '是', '不', '知', '道', '该', '说', '什', '么', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, may, give, up, soon, and, just, nap, instead, ., @end@]
GOLD: [@start@, 也, 许, 我, 会, 马, 上, 放, 弃, 然, 后, 去, 睡, 一, 觉, 。, @end@]
PRED: ['我', '一', '无', '四', '時', '可', '能', '早', '點', '兒', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, 'm, going, to, go, ., @end@]
GOLD: [@start@, 我, 要, 走, 了, 。, @end@]
PRED: ['我', '要', '去', '。']
////////////////////////////////////////////

SOURCE: [@start@, That, 's, MY, line, !, @end@]
GOLD: [@start@, 那, 是, 我, 的, 台, 词, ！, @end@]

In [59]:
vocab2 = Vocabulary.from_files("/content/drive/My Drive/Eng_Mandarin/data/tmp_v02.1/vocabulary")
model2 = SimpleSeq2Seq(vocab2, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)

def convert(s): 
    new = ""   
    for x in s: 
        new += x   
    return new 

with open("/content/drive/My Drive/Eng_Mandarin/data/tmp_v02.1/model.th", 'rb') as f:
  model2.load_state_dict(torch.load(f))

if CUDA_DEVICE > -1:
    model2.cuda(CUDA_DEVICE)

predictor2 = SimpleSeq2SeqPredictor(model2, reader)

for instance in itertools.islice(validation_dataset, 10):
  print('SOURCE:', instance.fields['source_tokens'].tokens)
  print('GOLD:', instance.fields['target_tokens'].tokens)
  print('PRED:', convert(predictor2.predict_instance(instance)['predicted_tokens']))
  #print('PRED:', predictor2.predict_instance(instance)['predicted_tokens'])
  print('////////////////////////////////////////////\n')

SOURCE: [@start@, I, have, to, go, to, sleep, ., @end@]
GOLD: [@start@, 我, 该, 去, 睡, 觉, 了, 。, @end@]
PRED: 我该睡睡觉。。
////////////////////////////////////////////

SOURCE: [@start@, I, just, do, n't, know, what, to, say, ., @end@]
GOLD: [@start@, 我, 就, 是, 不, 知, 道, 說, 些, 什, 麼, 。, @end@]
PRED: 我只是不知道本什什么。
////////////////////////////////////////////

SOURCE: [@start@, I, may, give, up, soon, and, just, nap, instead, ., @end@]
GOLD: [@start@, 也, 许, 我, 会, 马, 上, 放, 弃, 然, 后, 去, 睡, 一, 觉, 。, @end@]
PRED: 我许我们早上离下。后。。觉下。
////////////////////////////////////////////

SOURCE: [@start@, I, 'm, going, to, go, ., @end@]
GOLD: [@start@, 我, 要, 走, 了, 。, @end@]
PRED: 我要去了。
////////////////////////////////////////////

SOURCE: [@start@, That, 's, MY, line, !, @end@]
GOLD: [@start@, 那, 是, 我, 的, 台, 词, ！, @end@]
PRED: 那是我該环程！
////////////////////////////////////////////

SOURCE: [@start@, It, does, n't, surprise, me, ., @end@]
GOLD: [@start@, 这, 并, 不, 让, 我, 惊, 讶, 。, @end@]
PRED: 它个没打我惊讶。
///////////////////////